In [132]:
import os
import re
import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

'''from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
'''
from sklearn.feature_extraction.text import CountVectorizer

In [133]:
yelp_business = pd.read_json('../input/yelp_academic_dataset_business.json', lines=True)
yelp_business.fillna('NA', inplace=True)
# we want to make sure we only work with restaurants -- nothing else
restaurants = yelp_business[yelp_business['categories'].str.contains('Restaurants')]
print('Number of all businesses: ',yelp_business.shape[0])
print(f"Shape of restaurants dataset{restaurants.shape}")

Number of all businesses:  192609
Shape of restaurants dataset(59371, 14)


In [134]:
yelp_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,NA,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'Caters': 'True', 'NoiseLevel': 'u'loud'', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'False', 'BikeParking': 'False', 'Ambience': '{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}', 'HasTV': 'False', 'WiFi': 'u'no'', 'GoodForKids': 'True', 'Alcohol': 'u'full_bar'', 'RestaurantsAttire': 'u'casual'', 'RestaurantsGoodForGroups': 'True', 'RestaurantsDelivery': 'False'}",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported Food, Food, Chinese, Ethnic Food, Seafood",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'Wednesday': '9:0-0:0', 'Thursday': '9:0-0:0', 'Friday': '9:0-1:0', 'Saturday': '9:0-1:0', 'Sunday': '9:0-0:0'}",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'average'', 'RestaurantsDelivery': 'False', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}', 'Alcohol': 'u'beer_and_wine'', 'Caters': 'False', 'WiFi': 'u'no'', 'RestaurantsTakeOut': 'True', 'BusinessAcceptsCreditCards': 'True', 'Ambience': '{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'RestaurantsTableService': 'True', 'RestaurantsGoodForGroups': 'True', 'OutdoorSeating': 'False', 'HasTV': 'True', 'BikeParking': 'True', 'RestaurantsReservations': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsAttire': ''casual''}",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-21:30', 'Thursday': '17:30-21:30', 'Friday': '17:30-22:0', 'Saturday': '17:30-22:0', 'Sunday': '17:30-21:0'}",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",NA,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', 'Wednesday': '8:0-17:0', 'Thursday': '8:0-17:0', 'Friday': '8:0-17:0'}",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppointmentOnly': 'True', 'BusinessAcceptsCreditCards': 'True'}",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Services, Kitchen & Bath, Home & Garden, Water Heater Installation/Repair",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', 'Wednesday': '7:0-23:0', 'Thursday': '7:0-23:0', 'Friday': '7:0-23:0', 'Saturday': '7:0-23:0', 'Sunday': '7:0-23:0'}",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


In [135]:
restaurants.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'Caters': 'True', 'NoiseLevel': 'u'loud'', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'False', 'BikeParking': 'False', 'Ambience': '{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}', 'HasTV': 'False', 'WiFi': 'u'no'', 'GoodForKids': 'True', 'Alcohol': 'u'full_bar'', 'RestaurantsAttire': 'u'casual'', 'RestaurantsGoodForGroups': 'True', 'RestaurantsDelivery': 'False'}",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported Food, Food, Chinese, Ethnic Food, Seafood",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'Wednesday': '9:0-0:0', 'Thursday': '9:0-0:0', 'Friday': '9:0-1:0', 'Saturday': '9:0-1:0', 'Sunday': '9:0-0:0'}",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'average'', 'RestaurantsDelivery': 'False', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}', 'Alcohol': 'u'beer_and_wine'', 'Caters': 'False', 'WiFi': 'u'no'', 'RestaurantsTakeOut': 'True', 'BusinessAcceptsCreditCards': 'True', 'Ambience': '{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'RestaurantsTableService': 'True', 'RestaurantsGoodForGroups': 'True', 'OutdoorSeating': 'False', 'HasTV': 'True', 'BikeParking': 'True', 'RestaurantsReservations': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsAttire': ''casual''}",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-21:30', 'Thursday': '17:30-21:30', 'Friday': '17:30-22:0', 'Saturday': '17:30-22:0', 'Sunday': '17:30-21:0'}",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
11,2450 E Indian School Rd,"{'RestaurantsTakeOut': 'True', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'WiFi': 'u'no'', 'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'RestaurantsAttire': 'u'casual'', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsReservations': 'False', 'HasTV': 'False', 'Ambience': '{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': False}', 'Alcohol': 'u'none'', 'RestaurantsPriceRange2': '1', 'GoodForKids': 'True'}",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Tacos, Tex-Mex, Fast Food",Phoenix,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'Wednesday': '7:0-0:0', 'Thursday': '7:0-1:0', 'Friday': '7:0-1:0', 'Saturday': '7:0-1:0', 'Sunday': '7:0-0:0'}",1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ
13,5981 Andrews Rd,"{'RestaurantsPriceRange2': '2', 'BusinessAcceptsCreditCards': 'True', 'GoodForKids': 'True', 'RestaurantsDelivery': 'True', 'Alcohol': 'u'none'', 'OutdoorSeating': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsTakeOut': 'True', 'RestaurantsReservations': 'False', 'BikeParking': 'True', 'RestaurantsAttire': 'u'casual'', 'Ambience': 'None', 'BusinessParking': 'None'}",fweCYi8FmbJXHCqLnwuk8w,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,"{'Monday': '10:0-0:0', 'Tuesday': '10:0

Now we bring the reviews and perform some preprocessing on those reviews..

In [136]:
yelp_review_iter = pd.read_json('../input/yelp_academic_dataset_review.json', chunksize=100000, lines=True)

Because reviews are too big, we will read them in chunks, and make sure we delete reviews of places that are not in our list of businesses filtered earlier. Note here we choose 5 chunks, but we could have chosen any number (larger numbers will give MemoryError later on).

In [137]:
yelp_review = pd.DataFrame()
i=0
for df in yelp_review_iter:
    
    df = df[df['business_id'].isin(restaurants['business_id'])]
    print(df.shape)
    yelp_review = pd.concat([yelp_review, df])
    print(i)
    if i==70: break
    i=i+1

(63667, 9)
0
(63513, 9)
1
(63553, 9)
2


KeyboardInterrupt: 

In [ ]:
yelp_review.shape

Also make sure we only get businesses that already show up in our review list and delete the rest.

In [ ]:
import pickle
yelp_review.to_pickle("pickled_reviews.pickle")


In [ ]:
rest_reviews =pd.read_pickle("pickled_reviews.pickle")

In [ ]:
yelp_business = yelp_business[yelp_business['business_id'].isin(rest_reviews['business_id'])]

In [ ]:
print('Final businesses shape: ', yelp_business.shape)
print('Final review shape: ', rest_reviews.shape)

In [ ]:
rest_reviews.head()

In [ ]:
rest_reviews['funny'].describe()

In [ ]:
pd.options.display.max_seq_items = 2000
print(rest_reviews[rest_reviews['funny']==1290][['business_id', 'text']])

In [ ]:
# Check:
print( (rest_reviews['funny']>4).mean())
print(f"Number of funny reviews:{(rest_reviews['funny']>4).sum()}")
#print(rest_reviews['fun_bin'].mean())

In [ ]:
75269/4201684

In [ ]:
rest_reviews['fun_bin']=rest_reviews['funny'].apply(lambda x: 1 if x>4 else 0)

In [ ]:
print(rest_reviews['fun_bin'].mean())

#### Getting a df with funny reviews

In [ ]:
rest_reviews_fun = rest_reviews[rest_reviews['fun_bin']==1]

In [ ]:
rest_reviews_fun.shape

In [ ]:
rest_reviews_fun.drop_duplicates(subset= 'text', inplace=True)

In [ ]:
rest_reviews_fun.shape

### Sampling not funny reviews


In [ ]:
rest_reviews_not_fun = rest_reviews[rest_reviews['fun_bin']==0]

In [ ]:
idx = rest_reviews_not_fun.index.values

In [ ]:
len(idx)

In [ ]:
# Balancing the classes: getting the same number of not funny reviews as funny
#random_hotels = np.random.choice(neg_activity_df["hotel"].unique(), len(neg_activity_df))
random_idx = np.random.choice(idx,rest_reviews_fun.shape[0])

In [ ]:
len(random_idx)

In [ ]:
rest_reviews_not_fun = rest_reviews_not_fun.loc[random_idx,:].copy()

In [ ]:
rest_reviews_not_fun.shape

In [ ]:
reviews_final = pd.concat([rest_reviews_fun, rest_reviews_not_fun])

In [ ]:
reviews_final.shape

In [ ]:
reviews_final.to_csv("../balanced_reviews.csv")

In [ ]:
reviews_final = pd.read_csv("../balanced_reviews.csv", index_col=0)

In [ ]:
reviews_final.head()

In [ ]:
df_reviews = reviews_final[['funny','text', 'fun_bin']]

In [ ]:
pd.set_option('display.max_colwidth', -1)
df_reviews.head()

### Data pre-processing

#### Goals
- Keep punctuation
- Split by ".", "!" to account for misspeling (like "Hi!I went to...")

In [ ]:
!pip install spacymoji

In [ ]:
import pandas as pd
import numpy as np
from string import punctuation
import os
import spacy
import string
import re
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from spacymoji import Emoji

In [ ]:

re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

#nlp = spacy.load("en")
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

def clean_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
   # text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=\(\)]", " ", text) # keep punctuatuin, numnbers and letters
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " . ", text) #Add space to the dot
    text = re.sub(r"!", " ! ", text) #Add space to the exclamation sign
    text = re.sub(r":", " :", text) #Add space before : sign
    text = re.sub(r";", " ;", text) #Add space before ; sign
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    #text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    #text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # find emojis
    emoji_list = []
    '''
    for word in text.split():
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    emoji_list'''
    #text = text.split()

    return text

my_tok = spacy.load('en')
emoji = Emoji(my_tok)
my_tok.add_pipe(emoji, first=True)
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(clean_text(x))]

def remove_stop_words(tokens): return [tok for tok in tokens if tok not in spacy_stopwords]

In [ ]:
text = "I'm soooo excited!!!!!This is 10000% the best place on earth:))))) 😃..."

In [ ]:
text

In [ ]:
clean_text(text)

In [ ]:
spacy_tok(clean_text(text))

In [ ]:
text2 = "I also ordered a jade chicken quesadilla on the side.\n\nI'm gonna admit, this place looks kinda dirty. I don't think Arizona uses those health department letter grade system like California does, but if I were to just judge by how it looked inside, i'd give it a 'C' grade lol 😃"

In [ ]:
remove_stop_words(spacy_tok(clean_text(text2)))

### Building a vocabulary

In [ ]:
counts = Counter()
for sent in df_reviews['text']:
    try:
        counts.update(remove_stop_words(spacy_tok(sent)))
    except:
        pass

In [ ]:
counts

In [ ]:

len(counts)

In [ ]:
# Vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
df_reviews['len_text'] = df_reviews['text'].apply(lambda x: len(x.split()))

In [ ]:
# WHat is the 99% quantile of  length of the sentence?
df_reviews['len_text'].quantile(0.99)

In [ ]:
# WHat is the 95% quantile of  length of the sentence?
df_reviews['len_text'].quantile(0.95)

In [ ]:
# note that spacy_tok takes a while run it just once
def encode_sentence(sent, vocab2index, N=500, padding_start=True):
    "Encoding a sentence adding padding"
    x = remove_stop_words(spacy_tok(sent))
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc, l

In [ ]:
l1 = [encode_sentence(sent, vocab2index, N=600, padding_start=False) for sent in df_reviews['text'][:5]]

In [ ]:
l1

### Splitting into train and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
 X_train, X_valid, y_train, y_valid = train_test_split(df_reviews['text'], df_reviews['fun_bin'], test_size=0.2, random_state=42)

In [ ]:
X_train.reset_index(inplace=True, drop=True)
X_valid.reset_index(inplace=True, drop=True)

In [ ]:
y_train.reset_index(inplace=True, drop=True)

In [ ]:
y_valid.reset_index(inplace=True, drop=True)

In [ ]:
X_train[5]

### Writing a dataset

In [ ]:
class YelpDataset(Dataset):
    def __init__(self, df, y, N=500, padding_start=True):
        self.df = df
        self.X = [encode_sentence(sent, vocab2index, N, padding_start) for sent in self.df]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.X[idx]
        return x, s, self.y[idx]


In [ ]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)


#### Check training and validation datasets for empty sentences

In [ ]:
# training
neg=[]
i=0
for x,s,y in train_ds:
    if s <=0:
        neg.append(i)
    i+=1

In [ ]:
neg

In [ ]:
# validation
neg=[]
i=0
for x,s,y in valid_ds:
    if s <=0:
        neg.append(i)
    i+=1

In [ ]:
neg

In [ ]:
X_train[[111618, 118582]]

In [ ]:
train_ds[118582]

In [ ]:
train_ds[111618]

In [ ]:
X_train.shape

In [ ]:
X_train.drop([111618, 118582], inplace=True)

In [ ]:
X_train.shape

In [ ]:
X_train.reset_index(drop=True, inplace=True)

In [ ]:
y_train.drop([111618, 118582], inplace=True)

In [ ]:
y_train.shape

In [ ]:
y_train.reset_index(drop=True, inplace=True)

In [ ]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)

In [ ]:
train_ds[118582]

In [ ]:
len(y_valid)

In [ ]:
batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [ ]:
np.where(df_reviews.isna())

In [171]:
class LSTM_CNN(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTM_CNN,self).__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.5)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.conv_3 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=hidden_dim, out_channels=100, kernel_size=5)
        self.linear = nn.Linear(403, 1)        
        
    def forward(self, x, s):
        # sorting
        s, sort_index = torch.sort(s.float(), 0,descending=True) # s is the length of the sentence. Sort these lengths
        s = s.numpy().tolist() # 
        x_lstm = x[sort_index]
        x_lstm = self.embeddings(x_lstm)
        x_lstm = self.dropout(x_lstm)
        x_pack = pack_padded_sequence(x_lstm, s, batch_first=True) # We want LSTM to forget the padding, but in order to apply 
        #ordering mini batches withtin the model
        out_pack, (ht, ct) = self.lstm(x_pack) 
        #out_lstm = self.linear(ht[-1]) # Problem here is that output is not sorted!
        #out_lstm = torch.zeros_like(out_lstm).scatter_(0, sort_index.unsqueeze(1).cuda(), out_lstm)
        # scatter_ is undoing the sorting with the given sorting index
        # kind of sorting back with the original indexing
        x_cnn = self.embeddings(x)
        #print(x_cnn.size())
        x_cnn = x_cnn.transpose(1,2)
        #print(x_cnn.size())
        x3 = F.relu(self.conv_3(x_cnn))
        x4 = F.relu(self.conv_4(x_cnn))
        x5 = F.relu(self.conv_5(x_cnn))
        x3 = nn.MaxPool1d(kernel_size = 398)(x3)
        x4 = nn.MaxPool1d(kernel_size = 397)(x4)
        x5 = nn.MaxPool1d(kernel_size = 396)(x5)
        out_cnn = torch.cat([x3, x4, x5], 2)
        #print(out_cnn.size())
        out_cnn = out_cnn.view(out_cnn.size(0), -1)
        out = torch.cat([out_cnn, ht[-1]],1)
        out = self.dropout(out)
        #print(out.size())
        out = self.linear(out)
        return out 
        

In [174]:
def val_metrics(m, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        x = x.long().cuda()
        y = y.float().cuda().unsqueeze(1)
        y_hat = model(x,s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [ ]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long().cuda()
            y = y.float().cuda()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [ ]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [ ]:
vocab_size = len(words)
print(vocab_size)

In [172]:
model = LSTM_CNN(vocab_size, 103,103).cuda()

In [175]:
train_epocs(model, epochs=5, lr=0.01)

train loss 0.599 val loss 0.580 and val accuracy 0.705
train loss 0.553 val loss 0.685 and val accuracy 0.718
train loss 0.519 val loss 0.695 and val accuracy 0.714
train loss 0.486 val loss 0.890 and val accuracy 0.713
train loss 0.467 val loss 0.971 and val accuracy 0.701


In [176]:
save_model(model, '../model_lr0.01_5.pth')

In [177]:
train_epocs(model, epochs=5, lr=0.001)

train loss 0.440 val loss 1.139 and val accuracy 0.709
train loss 0.430 val loss 1.216 and val accuracy 0.709
train loss 0.424 val loss 1.365 and val accuracy 0.710
train loss 0.421 val loss 1.443 and val accuracy 0.709
train loss 0.418 val loss 1.598 and val accuracy 0.711


In [178]:
save_model(model, '../model_lr0.001_5.pth')

In [179]:
train_epocs(model, epochs=5, lr=0.0001)

train loss 0.416 val loss 1.556 and val accuracy 0.709
train loss 0.415 val loss 1.560 and val accuracy 0.708
train loss 0.415 val loss 1.582 and val accuracy 0.709
train loss 0.415 val loss 1.596 and val accuracy 0.709
train loss 0.414 val loss 1.606 and val accuracy 0.708


In [180]:
save_model(model, '../model_lr0.0001_5.pth')

In [181]:
train_epocs(model, epochs=5, lr=0.001)

train loss 0.415 val loss 1.711 and val accuracy 0.706
train loss 0.413 val loss 1.883 and val accuracy 0.709
train loss 0.411 val loss 1.890 and val accuracy 0.708
train loss 0.410 val loss 2.097 and val accuracy 0.709
train loss 0.409 val loss 2.175 and val accuracy 0.707


In [182]:
train_epocs(model, epochs=5, lr=0.01)

train loss 0.444 val loss 2.474 and val accuracy 0.703
train loss 0.473 val loss 2.496 and val accuracy 0.707
train loss 0.471 val loss 2.204 and val accuracy 0.700
train loss 0.459 val loss 2.442 and val accuracy 0.694
train loss 0.455 val loss 3.331 and val accuracy 0.707


In [183]:
train_epocs(model, epochs=5, lr=0.001)

train loss 0.436 val loss 3.113 and val accuracy 0.706
train loss 0.424 val loss 3.092 and val accuracy 0.703
train loss 0.419 val loss 3.218 and val accuracy 0.707
train loss 0.415 val loss 3.371 and val accuracy 0.706
train loss 0.414 val loss 3.437 and val accuracy 0.706
